In [1]:
#### implementation of the pytorch bert model
#!pip install huggingface-hub
#!pip install --upgrade tensorflow-gpu # for Python 3.n and GPU

#https://www.kaggle.com/dhruvag/bert-multi-class-classification-for-large-text

#https://www.philschmid.de/bert-text-classification-in-a-different-language

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import torch
import pandas as pd
import numpy as np
#model bert-base-german-dbmdz-cased https://github.com/dbmdz/berts


import pandas as pd
import numpy as np

'''
BEFORE RUNNING THIS NOTEBOOK MAKE SURE THO AHNGE 7ADJUST THE NAMES OF THE OUTPUT .CSV AND .PTH FILE FOR SAVING THE RESULTS AND TRAINED  MODEL

'''





In [ ]:
downsample = False

if downsample:
    # requires Dataframe to have two specific column names label and cat label, 
    # those are needed to assess the label distribution and will be dropped afterwards

    df = pd.read_csv('to_one_hot.csv',index_col=0)
    num_labels = len(df.iloc[:,1].unique())
    print(num_labels)
    y = pd.get_dummies(df.label, prefix='label')
    #print(y.head())

    ddf = pd.concat([df.iloc[:,0:3], y], axis=1)
    print(ddf.columns, ddf.shape,ddf.head())

    #print(ddf.columns)
    col_order = ddf.columns
    # nichts herunter sampeln!
    target_count = ddf.label.value_counts()

    print('Class 0:', target_count[0])
    print('Class 1:', target_count[1:])
    #print('Proportion:', round(target_count[0] / target_count[1:], 2), ': 1')

    target_count.plot(kind='bar', title='Count (target)');

    # https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets
    # Enthaöltung und nichts auf 250 herunter samplen
    # class 1 nichts hat 250, undersample class 0 und class 1
    # Divide by class
    if 'cat labels' in df.columns:

        df_class_0 = ddf[ddf['cat labels'] == 0]
        df_class_1 = ddf[ddf['cat labels'] == 1]
        df_class_A = ddf[ddf['cat labels'] > 1]
    else:
        df_class_0 = ddf[ddf['label'] == 'Nichts']
        df_class_1 = ddf[ddf['label'] == 'Enthaltung']
        df_class_A = ddf[~ddf['label'].str.contains( 'Nichts| Sonstiges')]

    df_class_0_under = df_class_0.sample(n=250)
    df_class_1_under = df_class_1.sample(n=250)
    df_sampled = pd.concat([df_class_0_under, df_class_1_under,df_class_A])
    print(df_sampled.shape)

    if 'cat labels' in df.columns:

        print('Random under-sampling:')
        print(df_sampled['cat labels'].value_counts())

        df_sampled['cat labels'].value_counts().plot(kind='bar', title='Count (target)');

        df = df_sampled.drop(['label', 'cat labels'], axis=1)

    else:

        print('Random under-sampling:')
        df_sampled['label'].value_counts().plot(kind='bar', title='Count (target)')
        df = df_sampled.drop(['label'], axis=1)

else:
    df = pd.read_csv('one_label_one_hot_nkm_open.csv',index_col=0)
    num_labels = len(df.columns)-1
    print(num_labels)



In [6]:

answers = df.iloc[:,0].values
print(answers[:5])

#str_labels = df.iloc[:,1].to_list()
labels = df.iloc[:,1:].values



input_ids = []
attention_masks = []
# line 13 to overcome HTTPError 403 rate limit exceeded
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True
BertTokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-german-dbmdz-cased') 


# For every sentence...
for sent in answers:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = BertTokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens=True,
                        max_length=512,
                        return_token_type_ids=False,
                        padding="max_length",
                        truncation=True,
                        return_attention_mask=True,
                        return_tensors='pt',
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
int_labels = torch.FloatTensor(labels)


# Print sentence 0, now as a list of IDs.
print('Original: ', answers[0])
print('Token IDs:', input_ids[0])

['Nichts' 'Nichts' 'Nichts' 'Nichts' 'Nichts']
[[0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
Original:  Nichts
Token IDs: tensor([ 102, 8009,  103,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,

Using cache found in /home/azureuser/.cache/torch/hub/huggingface_pytorch-transformers_master


In [7]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

print(input_ids.shape, attention_masks.shape, int_labels.shape)

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, int_labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))



# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


torch.Size([1420, 512]) torch.Size([1420, 512]) torch.Size([1420, 17])
1,278 training samples
  142 validation samples


In [53]:
#HELPING METHODS 
# Function to calculate the accuracy of our predictions vs labels
# TODO WRITE TUPLE OF LABEL AND PREDICTION TO CSV AS PREDICTION OUTPUT
import csv
import time
import datetime
import pandas as pd
import sys


def write_predictions(preds, labels):

    print("Diese Datei bitte umbenennen, sonst wird sie beim nächsten Lauf überschrieben.")

    with open('predictions.csv', 'w') as test_file:
        file_writer = csv.writer(test_file)

        for pred, label in zip(preds, labels):
            file_writer.writerow((pred, label))
    print(' File saved. ')



def flat_accuracy(preds, labels):
    # flatten multidimensional arrays to flat list
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    flat_acc = np.sum(pred_flat == labels_flat) / len(labels_flat)

    return flat_acc


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:

from transformers import BertForSequenceClassification, AdamW, BertConfig


BERTmodel = BertForSequenceClassification.from_pretrained(
    "bert-base-german-dbmdz-cased", # Use the 12-layer BERT model, with an uncased vocab.  bert-base-german-dbmdz-cased
    num_labels = num_labels, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)


# lr = args.learning_rate - default is 5e-5, our notebook had 2e-5'
# eps = args.adam_epsilon  - default is 1e-8.

optimizer = AdamW(BERTmodel.parameters(),lr = 2e-5, eps = 1e-8 )

from transformers import get_linear_schedule_with_warmup

#  The BERT authors recommend between 2 and 4. 

epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)


In [33]:
import random
import tensorflow as tf
import torch.nn.functional as F 


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# alternativly cuda for gpu
device = torch.device("cpu")

# store a number of quantities such as training and validation loss, 
# validation accuracy, and timings, raw predictions, the argmax of the prediction giving the index and hence the label and the true labels
training_stats = []

raw_predictions, predictions, true_labels = [], [], []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    BERTmodel.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 200 batches.
        if step % 200 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)#.argmax(axis=1)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device).argmax(axis=1)


        #print(b_input_ids.shape, b_input_mask.shape, b_labels.shape)
        

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        BERTmodel.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        model_outputs = BERTmodel(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)



        loss = model_outputs.loss
        #logits == probability distribution before normalization using softmax or sigmoid.
        logits = model_outputs.logits.detach().to(device)


        prediction = np.argmax(logits[0]) #tf.nn.sigmoid(logits) 
        label_ids = b_labels.numpy()

        raw_predictions.append(logits)
        predictions.append(prediction)
        true_labels.append(b_input_ids)
        
   
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        #print('loss. ', loss)
        total_train_loss += loss.item()


        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(BERTmodel.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    BERTmodel.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    eval_raw_predictions, eval_predictions, eval_true_labels = [], [], []


    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)#.argmax(axis=1)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device).argmax(axis=1)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax or sigmoid.
            model_val_outputs = BERTmodel(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        #which loss to use? FINE TUNING POTENTIAL
        #labels = torch.tensor([1,0]).unsqueeze(0)
        #outputs = BERTmodel(input_ids, attention_mask=b_input_mask)
        #loss = F.cross_entropy(labels, outputs[0])


        loss = model_val_outputs.loss
        logits = model_val_outputs.logits.detach().to(device)
        eval_prediction = np.argmax(logits[0]) #tf.nn.softmax(logits)


        eval_raw_predictions.append(logits)
        eval_predictions.append(eval_prediction)
        eval_true_labels.append(b_input_ids)
        
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        #total_eval_loss += loss

        # Move logits and labels to CPU
        logits = logits.detach().to(device).numpy()
        label_ids = b_labels.to(device).numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...

  Average training loss: 2.01
  Training epcoh took: 0:12:12

Running Validation...
  Accuracy: 0.26
  Validation Loss: 2.22
  Validation took: 0:00:15

======== Epoch 2 / 3 ========
Training...

  Average training loss: 2.14
  Training epcoh took: 0:08:45

Running Validation...
  Accuracy: 0.28
  Validation Loss: 2.18
  Validation took: 0:00:16

======== Epoch 3 / 3 ========
Training...

  Average training loss: 2.11
  Training epcoh took: 0:08:56

Running Validation...
  Accuracy: 0.28
  Validation Loss: 2.18
  Validation took: 0:00:17

Training complete!
Total training took 0:30:43 (h:mm:ss)


In [57]:
# fails with ValueError: Length of values (240) does not match length of index (10556)
#write_predictions_to_origin('one_label_one_hot_nkm_open.csv', predictions, labels)

write_predictions(predictions, labels)

Diese Datei bitte umbenennen, sonst wird sie beim nächsten Lauf überschrieben.


In [ ]:

torch.save({
        'model_state_dict': BERTmodel.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()}, 'one_hot_nkm_bert.pth')


https://www.kaggle.com/javaidnabi/toxic-comment-classification-using-bert/

https://medium.com/analytics-vidhya/multi-label-text-classification-using-transformers-bert-93460838e62b

https://towardsdatascience.com/building-a-multi-label-text-classifier-using-bert-and-tensorflow-f188e0ecdc5d
